<a href="https://colab.research.google.com/github/precessioncap/holocene-colab/blob/main/QuickStart_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QuickStarter to the tournament

##Basic step and workflow:

0 - Using this notebook

1 - Download data

2 - Explore data

3 - Choose and train a model

4 - Scoring

5 - Make prediction

6 - Submit

---

## 0 - Using this notebook 

To execute the cell press `shift+enter`. 

Follow the steps and login with your Google account.

In [ ]:
# Lib & Dependencies
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import requests
from scipy import stats

# 1 - Download data

We will provide you with two dataset

- Training_data will be use to train your model.
- Hackathon_data will be use to make your prediciton.


There is three target you need to provide prediction on: target_r, target_g, target_b.




In [ ]:
# Data Download (may take a few minutes depending on your network)
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv' 
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv' 

# Data for training
train_data = pd.read_csv(train_datalink_X)
# Data for which you will submit your prediction
test_data = pd.read_csv(hackathon_data_link)
# Targets to be predicted
train_targets = pd.read_csv(train_datalink_y)

#If you don't want to work with time serie
train_data = train_data.drop(columns=['Moons', 'id'])
test_data = test_data.drop(columns=['Moons', 'id'])

In [ ]:
display(train_data)
display(train_targets)
display(test_data)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29
0,0.00,0.00,0.00,0.25,0.25,0.25,0.00,0.25,0.50,0.50,0.50,0.25,0.50,1.00,0.00,0.00,0.25,0.25,0.25,1.00,0.00,0.25,0.25,1.00,0.75,0.25,0.50,0.50,0.50
1,0.50,0.00,0.25,0.00,0.25,0.25,0.75,0.50,0.00,0.00,0.00,0.00,0.00,0.75,0.00,0.25,0.25,0.25,0.25,0.50,0.25,0.00,0.50,0.00,1.00,0.00,0.25,1.00,1.00
2,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.00,0.00,0.00,0.75,0.25,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.75,0.00,1.00,1.00,0.50,0.50
3,0.25,0.25,0.25,0.25,0.25,0.25,0.50,0.75,0.25,0.25,0.25,0.25,0.25,0.75,0.50,0.50,0.25,0.25,0.50,0.50,0.50,0.25,0.25,0.50,0.75,0.00,0.00,1.00,0.75
4,0.50,1.00,1.00,1.00,1.00,1.00,0.50,0.25,1.00,1.00,1.00,1.00,1.00,0.75,0.75,0.75,0.75,0.75,1.00,0.75,1.00,1.00,1.00,1.00,0.50,1.00,1.00,0.75,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91070,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.50,0.25,0.50,0.50,0.25
91071,0.50,0.25,0.25,0.25,0.25,0.25,0.75,0.75,0.00,0.25,0.00,0.25,0.25,0.00,0.25,0.50,0.75,0.75,0.75,0.00,0.50,0.25,0.25,0.75,0.50,0.75,0.50,0.25,0.75
91072,1.00,0.25,0.50,0.25,0.00,0.00,1.00,0.75,0.00,0.50,0.25,0.00,0.00,0.00,0.50,1.00,0.50,0.50,0.50,0.25,0.50,0.00,0.00,1.00,1.00,0.50,0.50,0.50,0.25
91073,0.75,0.25,0.25,0.25,0.50,0.50,0.75,0.25,0.00,0.25,0.00,0.25,0.25,0.00,0.25,0.25,0.25,0.25,0.25,0.00,0.25,0.00,0.25,1.00,0.50,1.00,0.75,0.00,0.25


,target_r,target_g,target_b
0,0.00,0.75,0.75
1,0.75,0.25,0.25
2,0.75,0.50,1.00
3,0.75,0.00,0.25
4,0.00,0.00,0.25
...,...,...,...
91070,0.50,0.75,0.75
91071,0.25,0.50,0.00
91072,1.00,1.00,0.25
91073,0.00,1.00,0.75


,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29
0,1.00,0.75,0.50,0.75,0.75,1.00,0.50,0.75,0.75,1.00,0.50,0.75,1.00,0.00,0.00,1.00,0.50,0.50,0.50,0.00,0.75,0.75,0.75,0.00,0.00,0.50,1.00,0.50,0.75
1,0.00,0.75,0.75,0.75,0.50,0.50,0.50,0.50,0.75,0.75,0.75,0.50,0.50,0.00,0.50,0.25,0.50,0.50,0.50,0.00,0.25,0.50,0.25,0.75,0.50,0.25,0.50,0.25,0.25
2,1.00,1.00,1.00,1.00,1.00,1.00,0.75,0.50,1.00,1.00,1.00,1.00,1.00,0.00,0.50,1.00,1.00,1.00,1.00,0.00,0.50,1.00,0.75,1.00,0.75,0.50,0.25,0.50,0.25
3,1.00,0.25,0.25,0.25,0.25,0.25,0.50,0.50,0.50,0.25,0.25,0.25,0.50,0.00,0.25,0.50,0.25,0.25,0.50,0.00,0.50,0.50,0.25,0.50,0.00,0.25,0.25,0.75,0.75
4,1.00,1.00,1.00,1.00,0.75,0.75,0.75,0.50,1.00,1.00,1.00,0.75,0.75,0.00,0.75,1.00,0.75,0.75,0.75,0.00,0.50,0.75,0.75,0.50,0.00,0.00,0.50,0.00,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48532,0.75,0.50,0.50,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.50,0.50,0.50,0.25,0.75,0.25,0.00,0.00,0.00,0.50,0.00,0.50,0.50,0.75,1.00,0.50,0.00,0.00,0.00
48533,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.00,0.00,0.50,0.00,0.00,0.00,0.50,1.00,1.00,1.00,0.00,0.00
48534,1.00,0.75,0.75,0.75,0.75,0.75,0.75,0.00,0.75,0.75,0.75,0.75,0.75,0.25,0.25,0.75,0.00,0.00,0.25,0.50,0.00,0.25,0.25,0.25,0.75,0.00,0.75,0.00,0.00
48535,0.00,0.00,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.00,0.00,0.00,0.00,0.50,0.25,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.00,1.00,1.00,1.00,1.00,0.00,0.00


# 2 - Explore Data

Data processing is one of the most important part. Observe your data and prepare carefully what you will give to your model for training.

In [ ]:
display(train_data.describe())
display(train_targets.describe())

,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,Feature_9,Feature_10,Feature_11,Feature_12,Feature_13,Feature_14,Feature_15,Feature_16,Feature_17,Feature_18
count,91598.000000,91598.000000,91598.000000,91598.00000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000,91598.000000
mean,0.470324,0.463703,0.306944,0.45480,0.461148,0.457586,0.323694,0.466105,0.466091,0.466105,0.465870,0.450299,0.325154,0.454852,0.454816,0.483346,0.466629,0.490794
std,0.362686,0.364358,0.346557,0.36642,0.364976,0.365791,0.370961,0.363763,0.363770,0.363763,0.363818,0.367364,0.371667,0.366402,0.366402,0.359013,0.363635,0.355523
min,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.250000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.250000
50%,0.500000,0.500000,0.250000,0.50000,0.500000,0.500000,0.250000,0.500000,0.500000,0.500000,0.500000,0.500000,0.250000,0.500000,0.500000,0.500000,0.500000,0.500000
75%,0.750000,0.750000,0.500000,0.75000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000,0.750000
max,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


,target_r,target_g,target_b
count,91598.000000,91598.000000,91598.000000
mean,0.500000,0.500000,0.500000
std,0.353557,0.353557,0.353557
min,0.000000,0.000000,0.000000
25%,0.250000,0.250000,0.250000
50%,0.500000,0.500000,0.500000
75%,0.750000,0.750000,0.750000
max,1.000000,1.000000,1.000000


# 3 - Choose models

Crunch with originality!!! 👨🏻‍🏭

In [ ]:
def xg_boost_hackathon(data, target):
    X, y = data, target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    model = xgb.XGBRegressor(objective='reg:squarederror', max_depth=5, learning_rate=0.01, n_estimators=2000, n_jobs=-1, colsample_bytree=0.5)
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    scorer(y_test, pred)

    return model

# 4 - How we will compute your mean score

In [ ]:
def scorer(y_test, y_pred):
    score = (stats.spearmanr(y_test, y_pred)*100)[0]
    print('Score as calculated for the leader board (っಠ‿ಠ)っ {}'.format(score))

# Train your Models on targets

You can submit continious target if you want

In [ ]:
# Making prediction for target r
model_target_r = xg_boost_hackathon(train_data, train_targets.target_r)
# Making prediction for target g
model_target_g = xg_boost_hackathon(train_data, train_targets.target_g)
# Making prediction for target b
model_target_b = xg_boost_hackathon(train_data, train_targets.target_b)

Score as calculated for the leader board (っಠ‿ಠ)っ 0.0648480934975403
Score as calculated for the leader board (っಠ‿ಠ)っ 0.003304952555093535
Score as calculated for the leader board (っಠ‿ಠ)っ 0.03545533927861914


# 5 - Make prediction on the 3 targets

When you feel like your model is accurate enough it's time to predict the target and submit your results.

Repeat the operation on the three targets, concatenate the answers and submit.

**WARNING** 1/ Keep the raw order identical.

**WARNING** 2/ Be sure that your columns are named target_r, target-g and
 target_b.

**WARNING** 3/ Your prediction need to be between 0 and 1.

**WARNING** 4/ Don't submit constant values.

In [ ]:
prediction = pd.DataFrame()
prediction['target_r'] = model_target_r.predict(test_data)
prediction['target_g'] = model_target_g.predict(test_data)
prediction['target_b'] = model_target_b.predict(test_data)

In [ ]:
prediction

,target_r,target_g,target_b
0,0.487405,0.504527,0.407487
1,0.529782,0.556888,0.484716
2,0.442057,0.486263,0.514466
3,0.458316,0.457576,0.502802
4,0.481145,0.511609,0.556934
...,...,...,...
132895,0.598361,0.548681,0.576935
132896,0.519563,0.478392,0.514667
132897,0.501942,0.428857,0.410067
132898,0.519367,0.456818,0.451727


# 6 - Submit predictions

Past <u>your</u> API key here. You received it by email upon subscription and can find it on your leaderboard.


In [ ]:
API_KEY = "" # <- HERE

r = requests.post("https://tournament.datacrunch.com/api/submission",
    files = {
        "file": ("x", prediction.to_csv().encode('ascii'))
    },
    data = {
        "apiKey": API_KEY
    },
)

if r.status_code == 200:
    print("Submission submitted :)")
elif r.status_code == 423:
    print("ERR: Submissions are close")
    print("You can only submit during rounds eg: Friday 7pm GMT+1 to Sunday midnight GMT+1.")
    print("Or the server is currently crunching the submitted files, please wait some time before retrying.")
elif r.status_code == 422:
    print("ERR: API Key is missing or empty")
    print("Did you forget to fill the API_KEY variable?")
elif r.status_code == 404:
    print("ERR: Unknown API Key")
    print("You should check that the provided API key is valid and is the same as the one you've received by email.")
elif r.status_code == 400:
    print("ERR: The file must not be empty")
    print("You have send a empty file.")
elif r.status_code == 401:
    print("ERR: Your email hasn't been verified")
    print("Please verify your email or contact a cruncher.")
elif r.status_code == 409:
    print("ERR: Duplicate submission")
    print("Your work has already been submitted with the same exact results, if you think that this a false positive, contact a cruncher.")
    print("MD5 collision probability: 1/2^128 (source: https://stackoverflow.com/a/288519/7292958)")
elif r.status_code == 429:
    print("ERR: Too many submissions")
else:
    print("ERR: Server returned: " + str(r.status_code))
    print("Ouch! It seems that we were not expecting this kind of result from the server, if the probleme persist, contact a cruncher.")

ERR: API Key is missing or empty
Did you forget to fill the API_KEY variable?


In [ ]:
# Downloads data
from google.colab import files
with open("prediciton.csv", "wb") as f:
    f.write(prediction.to_csv().encode('ascii'))
files.download('prediciton.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
import pickle

# Export model as pickle
pickle.dump(model_target_r, open("model_target_r.model", "wb"))
files.download('model_target_r.model')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip freeze | grep "sklearn" 

sklearn==0.0
sklearn-pandas==1.8.0
